In [0]:
from pyspark.sql.functions import *

dbutils.widgets.text("source_schema" ,"" ,"source_schema")
source_schema = dbutils.widgets.get("source_schema")
print(f"source_schema - {source_schema}")

dbutils.widgets.text("backup_schema" ,"" ,"backup_schema")
backup_schema = dbutils.widgets.get("backup_schema")
print(f"backup_schema - {backup_schema}")

dbutils.widgets.text("source_table" ,"" ,"source_table")
source_table = dbutils.widgets.get("source_table")
print(f"source_table - {source_table}")

current_date = spark.sql("select current_date() as date")
date = current_date.withColumn("date", date_format(col("date"),"yyyyMMdd")).collect()[0][0]
backup_table = f"{source_table}_{date}"
print(f"backup_table - {backup_table}")

dbutils.widgets.text("duplicate_column" ,"" ,"duplicate_column")
duplicate_column = dbutils.widgets.get("duplicate_column")
print(f"duplicate_column - {duplicate_column}")

# If there is no filter condition (full backup), provide 1=1 or TRUE in filter_condition
dbutils.widgets.text("filter_condition" ,"" ,"filter_condition")
filter_condition = dbutils.widgets.get("filter_condition")
print(f"filter_condition - {filter_condition}")

source_schema - dw_xle_sz_delta_fdp
backup_schema - dw_xle_sz_archive
source_table - t_f_xle_ceded_claim_transaction_detail_as_is
backup_table - t_f_xle_ceded_claim_transaction_detail_as_is_20240221
duplicate_column - cctd_as_is_fact_key
filter_condition - src_sys_id in (15,23,26,80) and dw_row_active_ind = 'N'

In [0]:
partition_cols = spark.sql(f"DESC DETAIL {source_schema}.{source_table}").select('partitionColumns').collect()[0][0]
table_exists = spark.catalog._jcatalog.tableExists(backup_schema,backup_table)
if(table_exists):
  location = spark.sql(f"DESC DETAIL {backup_schema}.{backup_table}").select('location').collect()[0][0]
  backup_table_drop = f"DROP TABLE IF EXISTS {backup_schema}.{backup_table}"
  print(backup_table_drop)
  display(spark.sql(backup_table_drop))
  print(f"\nDropped {backup_schema}.{backup_table} table")
  dbutils.fs.rm(f'{location}', True)
  print(f"\nDropped {backup_schema}.{backup_table} path '{location}'")

bkp_db_lcoation = spark.sql(f"DESC DATABASE {backup_schema}").filter("database_description_item='Location'").select('database_description_value').collect()[0][0]
backup_table_create = f"""CREATE TABLE IF NOT EXISTS {backup_schema}.{backup_table}
USING DELTA
PARTITIONED BY ({', '.join(partition_cols)})
LOCATION '{bkp_db_lcoation}/{backup_table}'
AS SELECT * FROM {source_schema}.{source_table} WHERE {filter_condition}"""
print(backup_table_create)
display(spark.sql(backup_table_create))
print(f"\nCreated {backup_schema}.{backup_table}")

CREATE TABLE IF NOT EXISTS dw_xle_sz_archive.t_f_xle_ceded_claim_transaction_detail_as_is_20231031
USING DELTA
PARTITIONED BY (src_sys_id, dw_row_active_ind)
LOCATION 'abfs://standardizedzone@xlaeus2psedppdlgen2.dfs.core.windows.net/dw_xle_sz_archive/t_f_xle_ceded_claim_transaction_detail_as_is_20231031'
AS SELECT * FROM dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is WHERE src_sys_id in (15,23,26,80) and dw_row_active_ind = 'N'

num_affected_rows,num_inserted_rows


Created dw_xle_sz_archive.t_f_xle_ceded_claim_transaction_detail_as_is_20231031

In [0]:
source_backup_count_check = f"""SELECT '{source_schema}.{source_table}' AS Table, COUNT(*) AS Count 
                                FROM {source_schema}.{source_table} WHERE {filter_condition}
                                UNION ALL
                                SELECT '{backup_schema}.{backup_table}', COUNT(*) AS Count FROM {backup_schema}.{backup_table} 
                                WHERE {filter_condition} )"""
print(source_backup_count_check)
display(spark.sql(source_backup_count_check))

SELECT 'dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is' AS Table, Count FROM (
 SELECT 'dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is', COUNT(*) AS Count FROM dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is 
 WHERE src_sys_id in (15,23,26,80) and dw_row_active_ind = 'N'
 UNION ALL
 SELECT 'dw_xle_sz_archive.t_f_xle_ceded_claim_transaction_detail_as_is_20231031', COUNT(*) AS Count FROM dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is 
 WHERE src_sys_id in (15,23,26,80) and dw_row_active_ind = 'N' )

Table,Count
dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is,8625889512
dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is,8625889512


In [0]:
duplicate_count_check = f"""SELECT COUNT(*) AS duplicate_count FROM ( SELECT *, ROW_NUMBER() OVER (PARTITION BY {duplicate_column} ORDER BY dw_batch_id DESC, dw_last_update_dt DESC) rnk FROM {source_schema}.{source_table} WHERE {filter_condition} )a WHERE rnk>1"""
print(duplicate_count_check)
display(spark.sql(duplicate_count_check))

SELECT COUNT(*) AS duplicate_count FROM ( SELECT *, ROW_NUMBER() OVER (PARTITION BY cctd_as_is_fact_key ORDER BY dw_batch_id DESC, dw_last_update_dt DESC) rnk FROM dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is WHERE src_sys_id in (15,23,26,80) and dw_row_active_ind = 'N' )a WHERE rnk>1

duplicate_count
2916715242


In [0]:
delete_from_table = f"""DELETE FROM {source_schema}.{source_table} WHERE {filter_condition}"""
print(delete_from_table)
display(spark.sql(delete_from_table))

DELETE FROM dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is WHERE src_sys_id in (15,23,26,80) and dw_row_active_ind = 'N'

num_affected_rows
8625889512


In [0]:
columns = spark.sql(f"SELECT * FROM {source_schema}.{source_table}").columns
insert_into_table = f"""INSERT INTO {source_schema}.{source_table}
PARTITION ({', '.join(partition_cols)})
({', '.join(columns)})
SELECT {', '.join(columns)} FROM (
SELECT *, ROW_NUMBER() OVER (PARTITION BY {duplicate_column} ORDER BY dw_batch_id DESC, dw_last_update_dt DESC) rnk FROM {backup_schema}.{backup_table} WHERE {filter_condition} )a WHERE rnk=1"""
print(insert_into_table)
display(spark.sql(insert_into_table))

INSERT INTO dw_xle_sz_delta_fdp.t_f_xle_ceded_claim_transaction_detail_as_is
PARTITION (src_sys_id, dw_row_active_ind)
(cctd_as_is_fact_key, cctd_as_is_fact_key_parent, cctd_fact_key, cctd_fact_key_orig, cacctd_fact_key_orig, cctd_orig_amt, cctd_funct_amt, cctd_trans_amt, acctg_dt, acctg_pd_back_dated_ind, acctg_pd_future_dated_ind, chartf_uw_year, funct_curr_conv_dt, ledger_accident_year, loss_attach_dt, open_balance_trans_ind, sign_multiplier, src_creation_time_stamp_dt, tpa_claim_ind, trans_eff_dt, trans_entr_dt, acctg_pd_key, acctg_pd_key_actuarial_adjusted, acctg_pd_key_orig, acctg_pd_key_from_source, acctg_pd_key_when_created, b_ri_cont_layer_plcy_id, b_ri_cont_layer_plcy_key, bu_chartf_affiliate_id, bu_chartf_affiliate_key, cctd_facth_key, claim_id, claim_key, claim_line_id, claim_line_key, cmpl_id, cmpl_key, curr_orig_id, curr_orig_key, curr_settlement_id, curr_settlement_key, curr_trans_id, curr_trans_key, cvrg_id, cvrg_key, insd_loc_id, insd_loc_key, l_acctg_pd, l_claim_id, l_claim_key, l_cvrg_id, l_cvrg_key, l_plcy_id, l_plcy_key, l_ri_cont_layer_id, l_ri_cont_layer_key, lloyds_clas_cd_id, lloyds_clas_cd_key, lloyds_risk_id, lloyds_risk_key, loss_loc_id, loss_loc_key, mpl_id, mpl_key, obu_id, obu_key, org_gsr_id, org_gsr_key, org_ind_id, org_ind_key, org_le_id, org_le_key, org_le_intercompany_id, org_le_intercompany_key, org_lob_id, org_lob_key, org_por_id, org_por_key, org_uw_id, org_uw_key, party_broker_id, party_broker_key, party_ceded_broker_id, party_ceded_broker_key, party_claimant_id, party_claimant_key, party_insured_id, party_insured_key, party_insurer_id, party_insurer_key, party_reinsurer_id, party_reinsurer_key, plcy_id, plcy_key, plcy_cvrg_grp_id, plcy_cvrg_grp_key, plcy_cvrg_nbr, plcy_cvrg_nbr_id, plcy_cvrg_nbr_key, plcy_ledger_ri_type_id, plcy_ledger_ri_type_key, product_id, product_key, prog_id, prog_key, ri_cont_layer_id, ri_cont_layer_key, ri_cont_cvrg_grp_id, ri_cont_cvrg_grp_key, ri_cont_cvrg_nbr_id, ri_cont_cvrg_nbr_key, risk_loc_id, risk_loc_key, rpl_id, rpl_key, s2_prod_line_id, s2_prod_line_key, src_sys_key, ssr_bus_id, ssr_bus_key, ssr_trust_fund_id, ssr_trust_fund_key, trans_bus_type_key, trans_ent_type_key, trans_fin_type_id, trans_fin_type_key, trans_jrnl_type_key, trans_origin_type_key, cctd_facth_sz_key, curr_funct_id, curr_funct_key, cctd_src_trans_id, dw_batch_id, dw_create_dt, dw_last_update_dt, dw_row_end_dt, mi_curr_conv_dt, mi_curr_conv_type_cd, cctd_mi_eur_amt, cctd_mi_usd_amt, trans_type_admin_id, trans_type_admin_key, trans_migrated_ind, cctd_act_eur_amt, cctd_act_usd_amt, cctd_fin_eur_amt, cctd_fin_usd_amt, act_curr_conv_dt, act_curr_conv_type_cd, fin_account_id, fin_account_key, trans_type_actuarial_id, trans_type_actuarial_key, cctd_as_is_fact_key_orig, dw_last_update_dt_orig, bma_id, bma_key, rclass_id, rclass_key, src_sys_id, dw_row_active_ind)
SELECT cctd_as_is_fact_key, cctd_as_is_fact_key_parent, cctd_fact_key, cctd_fact_key_orig, cacctd_fact_key_orig, cctd_orig_amt, cctd_funct_amt, cctd_trans_amt, acctg_dt, acctg_pd_back_dated_ind, acctg_pd_future_dated_ind, chartf_uw_year, funct_curr_conv_dt, ledger_accident_year, loss_attach_dt, open_balance_trans_ind, sign_multiplier, src_creation_time_stamp_dt, tpa_claim_ind, trans_eff_dt, trans_entr_dt, acctg_pd_key, acctg_pd_key_actuarial_adjusted, acctg_pd_key_orig, acctg_pd_key_from_source, acctg_pd_key_when_created, b_ri_cont_layer_plcy_id, b_ri_cont_layer_plcy_key, bu_chartf_affiliate_id, bu_chartf_affiliate_key, cctd_facth_key, claim_id, claim_key, claim_line_id, claim_line_key, cmpl_id, cmpl_key, curr_orig_id, curr_orig_key, curr_settlement_id, curr_settlement_key, curr_trans_id, curr_trans_key, cvrg_id, cvrg_key, insd_loc_id, insd_loc_key, l_acctg_pd, l_claim_id, l_claim_key, l_cvrg_id, l_cvrg_key, l_plcy_id, l_plcy_key, l_ri_cont_layer_id, l_ri_cont_layer_key, lloyds_clas_cd_id, lloyds_clas_cd_key, lloyds_risk_id, lloyds_risk_key, loss_loc_id, loss_loc_key, mpl_id, mpl_key, obu_id, obu_key, org_gsr_id, org_gsr_key, org_in

num_affected_rows,num_inserted_rows
5709174270,5709174270


In [0]:
print(duplicate_count_check)
display(spark.sql(duplicate_count_check))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1395565820793915> in <module> 
 ----> 1 print ( duplicate_count_check ) 
 2 display ( spark . sql ( duplicate_count_check ) ) 

 NameError : name 'duplicate_count_check' is not defined